# Installing libraries

In [ ]:
!pip install langchain==0.0.225

In [ ]:
!pip install ctransformers==0.2.5

In [ ]:
!pip install sentence-transformers==2.2.2

In [37]:
!pip install pinecone-client==2.2.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
!pip install pypdf

# Importing all the modules

In [6]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Download the model from huggingface

In [11]:
!wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/raw/main/llama-2-7b-chat.ggmlv3.q4_0.bin

--2024-01-17 18:39:36--  https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/raw/main/llama-2-7b-chat.ggmlv3.q4_0.bin
Resolving huggingface.co (huggingface.co)... 13.35.166.69, 13.35.166.50, 13.35.166.36, ...
Connecting to huggingface.co (huggingface.co)|13.35.166.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 135 [text/plain]
Saving to: ‘llama-2-7b-chat.ggmlv3.q4_0.bin’

llama-2-7b-chat.ggm 100%[===================>]     135  --.-KB/s    in 0s      

2024-01-17 18:39:36 (133 MB/s) - ‘llama-2-7b-chat.ggmlv3.q4_0.bin’ saved [135/135]



# Unzip the data files

In [15]:
!unzip /content/test_data.zip

Archive:  /content/test_data.zip
  inflating: test_data/2-Aurelien-Geron-Hands-On-Machine-Learning-with-Scikit-Learn-Keras-and-Tensorflow_-Concepts-Tools-and-Techniques-to-Build-Intelligent-Systems-OReilly-Media-2019.pdf  


# Setting up the API KEYS

In [7]:
PINECONE_API_KEY = "a36608c4-ec9b-4c79-804d-ab3684ead68c"
PINECONE_API_ENV = "gcp-starter"

In [27]:
!pip install tqdm

In [63]:
def load_pdf(data):
    pdf_loader_kwargs={'autodetect_encoding': True}
    loader = DirectoryLoader(data,
                    glob="*.pdf",
            loader_cls=PyPDFLoader,show_progress=True)

    documents = loader.load()

    return documents

In [64]:
extracted_data = load_pdf("/content/test_data/")

100%|██████████| 1/1 [00:07<00:00,  7.37s/it]


In [65]:
len(extracted_data[100].page_content)

2203

In [66]:
extracted_data[100]

Document(page_content='Instead of a transformer, you can specify the string "drop"  if you\nwant the columns to be dropped. Or you can specify "pass\nthrough"  if you want the columns to be left untouched. By default,\nthe remaining columns (i.e., the ones that were not listed) will be\ndropped, but you can set the remainder  hyperparameter to any\ntransformer (or to "passthrough" ) if you want these columns to be\nhandled differently.\nIf you are using Scikit-Learn 0.19 or earlier, you can use a third-party library such as\nsklearn-pandas , or roll out your own custom transformer to get the same function‐\nality as the ColumnTransformer . Alternatively, you can use the FeatureUnion  class\nwhich can also apply different transformers and concatenate their outputs, but you\ncannot specify different columns for each transformer, they all apply to the whole\ndata. It is possible to work around this limitation using a custom transformer for col‐\numn selection (see the Jupyter notebook for

In [67]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [68]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 2335


In [69]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [70]:
embeddings = download_hugging_face_embeddings()

In [71]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [72]:
len(text_chunks)

2335

In [73]:
#Initializing the Pinecone
import pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="datascience-pilot"

In [74]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [78]:
def clean_malformed_surrogates(text, replacement=' '):
    """
    Clean malformed surrogate pairs in the text.
    :param text: The input text string.
    :param replacement: The string to replace malformed surrogates with.
    :return: Cleaned text string.
    """
    new_text = []
    skip_char = False
    for i, char in enumerate(text):
        if skip_char:
            skip_char = False
            continue

        if 0xD800 <= ord(char) <= 0xDBFF:  # High surrogate
            if i+1 < len(text) and 0xDC00 <= ord(text[i+1]) <= 0xDFFF:  # Correct low surrogate
                new_text.append(char + text[i+1])
                skip_char = True
            else:  # Malformed surrogate pair
                new_text.append(replacement)
        else:
            new_text.append(char)
    return ''.join(new_text)

# Use this function on your text data
cleaned_text_chunks = [clean_malformed_surrogates(t.page_content) for t in text_chunks]

# Then proceed with your embedding and Pinecone processes
docsearch = Pinecone.from_texts(cleaned_text_chunks, embeddings, index_name=index_name)


In [79]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What is Nonsaturating Activation Functions?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='These popular activation functions and their derivatives are represented in\nFigure 10-8 . But wait! Why do we need activation functions in the first place? Well, if\nyou chain several linear transformations, all you get is a linear transformation. For\nexample, say f( x) = 2 x + 3 and g( x) = 5 x - 1, then chaining these two linear functions\ngives you another linear function: f(g( x)) = 2(5 x - 1) + 3 = 10 x + 1. So if you don’t', metadata={}), Document(page_content='7“Self-Normalizing Neural Networks, " G. Klambauer, T. Unterthiner and A. Mayr (2017).\nFigure 11-3. ELU activation function\nIt looks a lot like the ReLU function, with a few major differences:\n•First it takes on negative values when z < 0, which allows the unit to have an\naverage output closer to 0. This helps alleviate the vanishing gradients problem,\nas discussed earlier. The hyperparameter α defines the value that the ELU func‐', metadata={}), Document(page_content='tor of bias term

In [81]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [82]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [88]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [89]:
# Clone only the required file to save space and time
!git lfs clone --depth 1 https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'Llama-2-7B-Chat-GGML'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 22 (delta 0), reused 22 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), 16.28 KiB | 2.03 MiB/s, done.
Exiting because of "interrupt" signal.
^C


In [90]:
%cd Llama-2-7B-Chat-GGML

/content/Llama-2-7B-Chat-GGML


In [91]:
!git lfs pull --include="llama-2-7b-chat.ggmlv3.q4_0.bin"


Exiting because of "interrupt" signal.
^C


In [93]:
llm=CTransformers(model="/content/Llama-2-7B-Chat-GGML/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

RuntimeError: Failed to create LLM 'llama' from '/content/Llama-2-7B-Chat-GGML/llama-2-7b-chat.ggmlv3.q4_0.bin'.

In [ ]:

qa=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])